In [6]:
import pandas as pd

/var/folders/ml/w7y812dd11v_q45g62cqxhh00000gn/T/ipykernel_78821/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
df = pd.read_csv("AirbnbData/Processed_Data/clean_tabular_data.csv")

In [39]:
label = "Price_Night"

features = df.select_dtypes([float, int]).drop("Unnamed: 0", axis=1)
Labels = features[label]
features

,guests,beds,bathrooms,Price_Night,Cleanliness_rating,Accuracy_rating,Communication_rating,Location_rating,Check-in_rating,Value_rating,amenities_count,bedrooms
0,2,1.0,1.0,105,4.6,4.7,4.3,5.0,4.3,4.3,13.0,1
1,3,3.0,0.0,92,4.3,4.7,4.6,4.9,4.7,4.5,8.0,1
2,4,2.0,1.5,52,4.2,4.6,4.8,4.8,4.8,4.7,51.0,1
3,2,1.0,1.0,132,4.8,4.9,4.9,4.9,5.0,4.6,23.0,1
4,4,3.0,1.0,143,5.0,4.9,5.0,4.7,5.0,4.7,32.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
824,4,2.0,1.5,240,4.9,5.0,5.0,5.0,4.9,4.8,33.0,2
825,2,1.0,1.0,78,4.8,5.0,4.9,4.9,5.0,4.9,54.0,1
826,4,2.0,1.5,113,4.8,5.0,5.0,5.0,5.0,4.8,38.0,2
827,6,3.0,2.0,80,4.7,4.8,5.0,5.0,5.0,4.7,24.0,2


In [10]:
numerical_features = ['guests', 'beds', 'bathrooms', 'Cleanliness_rating', 'Accuracy_rating',
                      'Communication_rating', 'Location_rating', 'Check-in_rating', 'Value_rating', 'amenities_count']
label = 'Price_Night'

In [12]:
df[numerical_features]

,guests,beds,bathrooms,Cleanliness_rating,Accuracy_rating,Communication_rating,Location_rating,Check-in_rating,Value_rating,amenities_count
0,2,1.0,1.0,4.6,4.7,4.3,5.0,4.3,4.3,13.0
1,3,3.0,0.0,4.3,4.7,4.6,4.9,4.7,4.5,8.0
2,4,2.0,1.5,4.2,4.6,4.8,4.8,4.8,4.7,51.0
3,2,1.0,1.0,4.8,4.9,4.9,4.9,5.0,4.6,23.0
4,4,3.0,1.0,5.0,4.9,5.0,4.7,5.0,4.7,32.0
...,...,...,...,...,...,...,...,...,...,...
824,4,2.0,1.5,4.9,5.0,5.0,5.0,4.9,4.8,33.0
825,2,1.0,1.0,4.8,5.0,4.9,4.9,5.0,4.9,54.0
826,4,2.0,1.5,4.8,5.0,5.0,5.0,5.0,4.8,38.0
827,6,3.0,2.0,4.7,4.8,5.0,5.0,5.0,4.7,24.0
